In [ ]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
strategy = tf.distribute.MirroredStrategy()

In [ ]:
from data_loader.create_dataset import create_tf_datasets

from models.attention_unet import AttentionUNet3D
from models.unet import UNet3D

from trainer.train import train_model

import numpy as np


# BC Dataset

In [ ]:
train_dir = 'data/BC/training_data/'

train_dataset, val_dataset = create_tf_datasets(
    train_dir, 
    percent_val=0.2, 
    patch_shape=(64, 64, 64), 
    patch_step=64,
    random_state = 42
)

In [ ]:
with strategy.scope():
    train_model(
        train_dataset = train_dataset, 
        val_dataset = val_dataset, 
        model = UNet3D().build_model(), 
        optimizer = 'adam', 
        loss = 'binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall'],
        epochs = 50,
        batch_size = 16, 
        filename="bc_unet3d.keras",
        model_name="bc_unet3d"
    )

In [ ]:
with strategy.scope():
    train_model(
        train_dataset = train_dataset, 
        val_dataset = val_dataset, 
        model = AttentionUNet3D().build_model(), 
        optimizer = 'adam',
        loss = 'binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall'],
        epochs = 50,
        batch_size = 16,
        filename="bc_attentionunet3d.keras",
        model_name="bc_attentionunet3d"
    )

# BC Augmented Dataset

In [ ]:
train_dir = 'data/BC/training_data_augm/'

train_dataset, val_dataset = create_tf_datasets(
    train_dir, 
    percent_val=0.2, 
    patch_shape=(64, 64, 64), 
    patch_step=64,
    random_state = 42
)

In [ ]:
with strategy.scope():
    train_model(
        train_dataset = train_dataset, 
        val_dataset = val_dataset, 
        model = UNet3D().build_model(), 
        optimizer = 'adam', 
        loss = 'binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall'],
        epochs = 50,
        batch_size = 16, 
        filename="bc_augmented_unet3d.keras",
        model_name="bc_augmented_unet3d"
    )

In [ ]:
with strategy.scope():
    train_model(
        train_dataset = train_dataset, 
        val_dataset = val_dataset, 
        model = AttentionUNet3D().build_model(), 
        optimizer = 'adam',
        loss = 'binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall'],
        epochs = 50,
        batch_size = 16,
        filename="bc_augmented_attentionunet3d.keras",
        model_name="bc_augmented_attentionunet3d"
    )

# Sinusoid Dataset

In [ ]:
train_dir = 'data/Sinusoids/training_data/'

train_dataset, val_dataset = create_tf_datasets(
    train_dir, 
    percent_val=0.2, 
    patch_shape=(64, 64, 64), 
    patch_step=64,
    random_state = 42
)

In [ ]:
with strategy.scope():
    train_model(
        train_dataset = train_dataset, 
        val_dataset = val_dataset, 
        model = UNet3D().build_model(), 
        optimizer = 'adam', 
        loss = 'binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall'],
        epochs = 50,
        batch_size = 16, 
        filename="sinusoids_unet3d.keras",
        model_name="sinusoids_unet3d"
    )

In [ ]:
with strategy.scope():
    train_model(
        train_dataset = train_dataset, 
        val_dataset = val_dataset, 
        model = AttentionUNet3D().build_model(), 
        optimizer = 'adam',
        loss = 'binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall'],
        epochs = 50,
        batch_size = 16,
        filename="sinusoids_attentionunet3d.keras",
        model_name="sinusoids_attentionunet3d"
    )

# Sinusoids Augmented Dataset

In [ ]:
train_dir = 'data/Sinusoids/training_data_augm/'

train_dataset, val_dataset = create_tf_datasets(
    train_dir, 
    percent_val=0.2, 
    patch_shape=(64, 64, 64), 
    patch_step=64,
    random_state = 42
)

In [ ]:
with strategy.scope():
    train_model(
        train_dataset = train_dataset, 
        val_dataset = val_dataset, 
        model = UNet3D().build_model(), 
        optimizer = 'adam', 
        loss = 'binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall'],
        epochs = 50,
        batch_size = 16, 
        filename="sinusoids_augmented_unet3d.keras",
        model_name="sinusoids_augmented_unet3d"
    )

In [ ]:
with strategy.scope():
    train_model(
        train_dataset = train_dataset, 
        val_dataset = val_dataset, 
        model = AttentionUNet3D().build_model(), 
        optimizer = 'adam',
        loss = 'binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall'],
        epochs = 50,
        batch_size = 16,
        filename="sinusoids_augmented_attentionunet3d.keras",
        model_name="sinusoids_augmented_attentionunet3d"
    )